In [3]:
# from openai import OpenAI

# # apikey.txt 파일에서 API 키를 읽습니다.
# with open("./bigdata/apikey.txt", "r") as file:
#     api_key = file.read().strip()

# # 읽은 API 키를 사용하여 클라이언트를 초기화합니다.
# client = OpenAI(api_key=api_key)

# stream = client.chat.completions.create(
#     model="gpt-4",
#     messages=[{"role": "user", "content": "Say this is a test"}],
#     stream=True,
# )

# for chunk in stream:
#     if chunk.choices[0].delta.content is not None:
#         print(chunk.choices[0].delta.content, end="")


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-mW8Lt***************************************gS6Q. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [2]:
제가 질문 과 답변을 드릴테니 답변에서 질문-답변 쌍을 만들어주세요. 질문은 이미 주어진 질문을 제외하고 답변을 보고 새로 생성해야합니다. 예를 들어서

번호1:
질문1:
철골구조는 어떤 방식이 있나요?
질문2:
철골구조의 다양한 방식이 무엇인가요?
답변1:
철골구조는 일반철골구조와 경량철골구조가 있습니다.
답변2:
철골구조는 일반철골구조와 경량철골구조가 있습니다. 일반철골구조는 주로 대형 건물이나 다층 주거 건물에 사용되며, 경량철골구조는 주로 주택이나 작은 상업 시설에 사용됩니다. 이 두 구조 방식은 건물의 규모와 용도에 따라 선택되어 적용됩니다.

이렇게 주어지면 질문1과 2에 속하지 않지만 답변2에는일반철골구조와 경량철골구조에 대한 설명이 있습니다. 즉 일반철골구조와 경량철골구조에 대해서 묻는 질문과 이에대한 답변을 생성할 수 있습니다. 예시 답변을 보여드리겠습니다.


생성번호1:
질문1:
일반철골구조는 어떤 건물 형태에 사용되나요?
답변1:
일반철골구조는 주로 대형 건물이나 다층 주거 건물에 사용됩니다.

생성번호2:
질문2:
경량철골구조는 어떤 건물 형태에 사용되나요?
답변2:
경량철골구조는 주로 주택이나 작은 상업 시설에 사용됩니다.

한 번호에 속한 두 질문에 대해서 답을 하는 네개의 답변을 드리겠습니다. 답변을 보고 주어진 질문과 다른 새로운 질문을 만들지 못하면 만들지 않아도 좋습니다. 또한, 답변에서 새로운 질문을 여러개 만들 수 있다면 여러개 만들어도 좋습니다. 당신은 네개의 답변을 그저 통합하는게 아니라 답변에서 새로운 질문을 만들고 그 질문에 답을 하는 답변을 추출해서 질문, 답변 쌍을 만드는 것입니다.  다음은 당신이 해줘야 되는 질문,답변을 드리겠습니다.



SyntaxError: invalid syntax (311968507.py, line 1)

In [5]:
# prompt generation

import pandas as pd
def read_input(path):
    with open("bigdata/augmentation_org.txt", "w") as file:
        data = pd.read_csv(path)
        input_list = []
        cnt = 0
        for _, row in data.iterrows():
            cnt+=1
#         for q_col in ['질문_1', '질문_2']:
#             for a_col in ['답변_1', '답변_2', '답변_3', '답변_4', '답변_5']:
            file.write(f"이어서 드리겠습니다. 답변에서 새로운 질문, 답변쌍을 추출하기 어려우면 추출하지 않아도 되고 여러개를 추출할 수 있으면 여러개를 추출해야합니다. 한 번호당 생성번호는 0개에서 5개까지 추출가능합니다.\n")
            file.write(f"번호{cnt}:\n")
            file.write(f"질문1:\n")
            file.write(row['질문_1'] + "\n")
            file.write(f"질문2:\n")
            file.write(row['질문_2'] + "\n")
            file.write(f"답변1:\n")
            file.write(row['답변_1'] + "\n")
            file.write(f"답변2:\n")
            file.write(row['답변_2'] + "\n")
            file.write(f"답변3:\n")
            file.write(row['답변_3'] + "\n")
            file.write(f"답변4:\n")
            file.write(row['답변_4'] + "\n\n\n")
read_input("bigdata/nobanmal/train.csv")     

In [127]:
# data preprocessing
# 파일 열기
import re
questions = []
answers = []

In [129]:



def read_aug(questions,answers,num):
    with open(f'./bigdata/augs_preprocessing/aug{num}.txt', 'r', encoding='utf-8') as file:

        reading_mode = None  # 'question', 'answer' 또는 None
        current_question = ""
        current_answer = ""

        for line in file:
            # 질문 시작 확인


            if '질문' in line and ':' in line:
                if current_question:  # 이미 질문이 있으면 저장
                    questions.append(current_question)
                    current_question = ""
                reading_mode = 'question'
                continue

            # 질문 끝 확인
            if ('?' in line or '.' in line) and reading_mode == 'question':
                current_question += line.strip()
                questions.append(current_question)
                current_question = ""
                reading_mode = None
                continue

            # 답변 시작 확인
            if '답변' in line and ':' in line:
                if current_answer:  # 이미 답변이 있으면 저장
                    answers.append(current_answer)
                    current_answer = ""
                reading_mode = 'answer'
                continue

            # 답변 읽기
            if reading_mode == 'answer':
                if not line.strip() or 'GPT' in line or re.search(r'번호\d', line):  # 답변 끝 조건
                    if current_answer:  # 답변이 있으면 저장
                        answers.append(current_answer.strip())
                        current_answer = ""
                    reading_mode = None
                    continue
                else:
                    current_answer += line

In [130]:
num_arr = []
for i in range(1,17):
    if i<10:
        read_aug(questions,answers,"0"+str(i))
    else:
        read_aug(questions,answers,str(i))
        


In [131]:
assert len(questions) == len(answers)

In [132]:
# 결과 출력

# for q, a in zip(questions, answers):
#     print("Q:", q)
#     print("A:", a)
#     print("---")


In [133]:
print(questions[-1])

벽체 결로 방지를 위해 단열재를 미실하게 시공하는 것의 중요성은 무엇인가요?


In [134]:
print(answers[-1])

벽체 결로 방지를 위해 단열재를 미실하게 시공하는 것의 중요성은 결로가 생기는 벽체의 표면 온도를 노점온도 이상으로 유지하여 결로 현상을 방지하고, 실내의 습도 문제를 최소화하기 위함입니다. 노점온도 이상으로 벽체의 표면 온도를 유지함으로써 습기가 벽체 표면에 응축되는 것을 방지할 수 있습니다.


In [135]:
import csv

# CSV 파일 생성 및 데이터 쓰기
with open('./bigdata/augs_preprocessing/qa_aug.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # CSV 파일의 헤더 작성
    writer.writerow(['Question', 'Answer'])
    
    # 질문과 답변 쌍을 파일에 쓰기
    for question, answer in zip(questions, answers):
        writer.writerow([question, answer])

print('CSV 파일이 생성되었습니다.')


CSV 파일이 생성되었습니다.
